# Connecting to Foursquare and Yelp APIs 
For each of the bike stations in Part 1, Foursquare and Yelp APIs used to retrieve information for the selected POIs:

Selected POIs are including following groups:
* Entertainment (Museum, Cinema, ...) 
* Education (School, University, College, ...) 
* Dining (Restuerant, Coffee shop, Fastfood, ...) 
* Park 
* Shopping (Shopping mall, store, grocery, ...)
* Metro_station
* Library
* Financial institution (Bank, ATM, Credit Union)


In [2]:
# imports
import pandas as pd
import os
import requests
import json
from pandas import json_normalize

# Environment variable
os.environ['FOURSQUARE_API_KEY'] = <API_KEY>
os.environ['YELP_API_KEY'] = <API_KEY>

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [26]:
# Function get_venues_4square sends request to FourSquare
def get_venues_4square( categories , latitude , longitude , radius , limit):
    
    # Make url
    url = "https://api.foursquare.com/v3/places/search?categories={}&ll={},{}&radius={}&limit={}".format(
                categories,
                latitude,
                longitude,
                radius,
                limit)

    # Create dictionary for headers
    headers = {"Accept": "application/json"}
    api_key = os.environ['FOURSQUARE_API_KEY']
    headers['Authorization'] = api_key

    # Send Request
    result = requests.get(url, headers=headers)
    
    # Compute number of places
    d=result.json()
    number_of_places = len(json_normalize(d['results']))

    return number_of_places


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# This code sends request to FourSquare and gets number of places for different categories and bike stations 

# Read api_key from OS environment variable
api_key    = os.environ['FOURSQUARE_API_KEY']
place_radius = 1000
place_limit = 50

# Load CityBikes data, saved in CSV file
df_citybikes = pd.read_csv("data/city_bikes.csv" , encoding = "unicode_escape")

# Point of interest (POI), based on FourSquare categories ( https://location.foursquare.com/places/docs/categories )
# List of selected categories
poi_name=[ 'entertainment' , 'education' , 'dining' , 'park' ,'shopping','metro_station'                         ,'library'     ,'bank','atm','creditunion' ]
poi_catid=[ '10001,10002,10004' , '12009,12013' , '13071,13072,13035,13145' , '16032,16036','17114,17115','19046','12080,12031','11046','11044','11045']

#Initializing foursquare dataframe
header = ['uid'] + poi_name
lst = len( df_citybikes ) * [ len( header ) * [0] ]
df_4square = pd.DataFrame( lst , columns = header , dtype = int )    

# Request Number Of All Place Category For All CityBikes Latitudes and Longitudes
for i in range(len( df_citybikes )):
    df_4square['uid'][i] = df_citybikes['uid'][i]   
    for j in range(len(poi_name)):
        df_4square[poi_name[j]][i] = get_venues_4square(poi_catid[j] , 
                                                        df_citybikes['latitude'][i] , 
                                                        df_citybikes['longitude'][i] , 
                                                        place_radius , 
                                                        place_limit)
       


Put your parsed results into a DataFrame

In [5]:
# Save results to CSV file
df_4square.to_csv('data/FourSquare.csv' , sep = ',' , encoding = 'utf-8',index=False)

# Show result
df_4square

,uid,entertainment,education,dining,park,shopping,metro_station,library,bank,atm,creditunion
0,7303,3,13,20,13,0,0,2,7,1,5
1,7301,8,24,22,13,2,0,2,17,2,12
2,7309,2,12,17,13,0,0,2,9,1,7
3,7045,37,50,50,25,8,6,12,50,46,50
4,7044,23,50,50,29,10,4,14,50,25,50
...,...,...,...,...,...,...,...,...,...,...,...
648,7521,20,46,37,14,2,2,1,20,3,15
649,7757,47,50,50,29,6,1,8,50,28,50
650,7311,15,47,40,23,1,3,9,47,13,32
651,1405,36,50,50,25,9,5,36,50,28,50


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [35]:
# Function get_venues_yelp sends request to Yelp
def get_venues_yelp( categories , latitude , longitude , radius , limit):
    
    # Make url
    url = "https://api.yelp.com/v3/businesses/search?latitude={}&longitude={}&term={}&radius={}&sort_by=best_match&limit={}".format(
                    latitude,
                    longitude,
                    categories,
                    radius,
                    limit)
   
    # Create dictionary for headers
    headers = {
    "accept": "application/json",
    "Authorization": "Bearer "+api_key}

    # Send Request
    result = requests.get(url , headers = headers)
    
    # Determine number of places
    d=result.json()
    number_of_places = d['total']

    return number_of_places

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# This code sends request to Yelp and gets number of places for different categories and bike stations 

# Read api_key from OS environment variable
api_key    = os.environ['YELP_API_KEY']
place_radius = 1000
place_limit = 50

# Load CityBikes data, saved in CSV file
df_citybikes = pd.read_csv("data/city_bikes.csv",encoding="unicode_escape")

# Point of interest (POI)
# List of selected categories
poi_name=['entertainment','education','dining','park','shopping','metro_station','library','financial']

#Initializing foursquare dataframe
header =['uid']+poi_name
lst = len(df_citybikes)*[len(header)*[0]]
df_yelp = pd.DataFrame(lst, columns =header, dtype = int)    

# Request Number Of All Place Category For All CityBikes Latitudes and Longitudes
for i in range(len(df_citybikes)):
    df_yelp['uid'][i]=df_citybikes['uid'][i]   
    for j in range(len(poi_name)):
        df_yelp[poi_name[j]][i]=get_venues_yelp(poi_name[j],
                                                df_citybikes['latitude'][i],
                                                df_citybikes['longitude'][i],
                                                place_radius,
                                                place_limit)
       


Put your parsed results into a DataFrame

In [7]:
# Save results to CSV file
df_yelp.to_csv('data/Yelp.csv' , sep = ',' , encoding = 'utf-8',index=False)

# Show result
df_yelp 

,uid,entertainment,education,dining,park,shopping,metro_station,library,financial
0,7303,7,6,48,8,36,0,1,2
1,7301,2,8,46,4,55,0,2,5
2,7309,7,5,42,8,37,0,1,1
3,7045,72,88,434,29,392,9,11,128
4,7044,34,58,369,22,331,3,12,62
...,...,...,...,...,...,...,...,...,...
648,7521,4,11,58,5,104,0,1,8
649,7757,62,32,383,25,234,2,3,77
650,7311,22,35,184,15,174,1,7,39
651,1405,58,82,444,33,371,5,22,94


# Comparing Results

##### Which API provided you with more complete data? Provide an explanation. 
According to FourSquare and Yelp descriptions and details, FourSquare data is categorized into many major and minor categories, but Yelp provides them in large categories. 
On the other side, the number of venues is limited to 50 by FourSquare. So FourSquare does not provide the correct value for high among.
So I decided to use Yelp information.

# Get the top 10 restaurants according to their rating
In this section restaurants and rates are retrieved from Yelp and the top 10 restaurants according to their rating are listed.

In [10]:
# This code gets the top 10 restaurants according to their rating in Yelp

# Function get_venues_yelp_rate sends request to Yelp and returns restaurants info including rate
def get_venues_yelp_rate( categories , latitude , longitude , radius , limit):

    # Make url
    url = "https://api.yelp.com/v3/businesses/search?latitude={}&longitude={}&term={}&radius={}&sort_by=best_match&limit={}".format(
                    latitude,
                    longitude,
                    categories,
                    radius,
                    limit)
   
    # Create dictionary for headers
    headers = {
    "accept": "application/json",
    "Authorization": "Bearer "+api_key}

    # Send Request
    result = requests.get(url , headers = headers)
    return result.json()

#======================================================

api_key    = os.environ['YELP_API_KEY']
place_radius = 1000
place_limit = 50

# Load CityBikes data, saved in CSV file
df_citybikes = pd.read_csv("data/citybikes.csv",encoding="unicode_escape")

# list of categories for listing rate
poi_name=['dining']

lst=[]

# Request Number Of All Place Category For All CityBikes Latitudes and Longitudes
for i in range(len(df_citybikes)):
    for j in range(len(poi_name)):
        d=get_venues_yelp_rate('dining',
                        df_citybikes['latitude'][i],
                        df_citybikes['longitude'][i],
                        place_radius,
                        place_limit)
        dd=d['businesses']
        for i in dd:
            address=i['location']['display_address']
            full_address=''
            for j in address: 
                full_address=full_address+j+','
            full_address=full_address[0:len(full_address)-1]
            new_rest={'name':i['name'],
                     'full_address':full_address,
                     'rate':i['rating']}
            if (new_rest not in lst):
                lst=lst+[new_rest]

# Convert to dataframe
df= pd.DataFrame(lst)

# sort rate in descending order
df.sort_values(by=['rate'], inplace=True, ascending=False)


In [16]:
# Save results to CSV file
df.to_csv('data/Yelp - Rates.csv' , sep = ',' , encoding = 'utf-8',index=False)

# Show result
df.iloc[:,2:].head(10)

,name,full_address,rate
0,Casamiento,"787 Dupont Street,Toronto, ON M6G 1Z5,Canada",5.0
1,Ohiru Cafe,"370 College Street,Toronto, ON M5T 1S6,Canada",5.0
2,YES Meal: 2 Can Dine 4 9.99 With Aunty Lucy's,"296 Brunswick Ave,Toronto, ON M5S 2M7,Canada",5.0
3,Kingyo Fisherman's Market,"51B Winchester Street,Toronto, ON M4X 1A8,Canada",5.0
4,Chandelier Dine & Lounge Bar,"1993 Yonge Street,Toronto, ON M4S 1Z8,Canada",5.0
5,Royal Canadian Military Institute,"426 University Avenue,Toronto, ON M5G 1S9,Canada",5.0
6,Domino's Pizza,"1131 Queen Street E,Toronto, ON M4M 1K9,Canada",5.0
7,Larry's Folly,"1462 Queen Street W,Toronto, ON M6K 1M2,Canada",5.0
8,Noon Eatery,"3190 Yonge Street,Toronto, ON M4N 2L1,Canada",5.0
9,Hawker,"291 Augusta Ave,Toronto, ON M5T 1N9,Canada",5.0
